In [186]:
import requests
import json
import pandas as pd
from scipy import stats


In [187]:
df_trump_2018 = pd.read_json('condensed_2018.json')
df_trump_2017 = pd.read_json('condensed_2017.json')
frames = [df_trump_2018, df_trump_2017]
df_trump = pd.concat(frames)

In [188]:
df_trump_count = df_trump
df_trump_count['date'] = df_trump['created_at'].dt.date
df_trump_count = df_trump_count.groupby(by='date').count()

In [189]:
df_trump_count['count']= df_trump_count['created_at']
df_trump_count['avg_favorites_per_day'] = df_trump.groupby(by='date').sum()['favorite_count']
df_trump_count['avg_favorites_per_day'] = df_trump_count['avg_favorites_per_day']/df_trump_count['count']
df_trump_count = df_trump_count[['count','avg_favorites_per_day']]

In [190]:
df_trump_approval = pd.read_csv('approval_topline.csv')
df_trump_approval = df_trump_approval.loc[df_trump_approval['subgroup'] == 'Voters']
df_trump_approval = df_trump_approval[['approve_estimate', 'disapprove_estimate', 'timestamp']]
df_trump_approval.timestamp = pd.to_datetime(df_trump_approval.timestamp)
df_trump_approval.timestamp = df_trump_approval.timestamp.dt.date

In [191]:
df_trump_approval.head()

,approve_estimate,disapprove_estimate,timestamp
0,42.81609,53.54470,2019-09-11
3,42.54695,53.41855,2019-09-10
8,42.62912,53.25327,2019-09-09
9,42.59635,53.27431,2019-09-08
12,42.64939,53.15034,2019-09-07


In [192]:
df_trump_count.head()

,count,avg_favorites_per_day
date,,
2017-01-01,6,21038.333333
2017-01-02,7,66845.285714
2017-01-03,10,57242.700000
2017-01-04,10,58512.900000
2017-01-05,6,70167.666667
